In [349]:
import numpy as np
with open("../data/day16.txt", "r") as f:
    input = f.read()

hex = input
print(hex)

bits = bin(int(hex, 16))[2:].zfill(4*len(hex))
#print(bits)

0052E4A00905271049796FB8872A0D25B9FB746893847236200B4F0BCE5194401C9B9E3F9C63992C8931A65A1CCC0D222100511A00BCBA647D98BE29A397005E55064A9DFEEC86600BD002AF2343A91A1CCE773C26600D126B69D15A6793BFCE2775D9E4A9002AB86339B5F9AB411A15CCAF10055B3EFFC00BCCE730112FA6620076268CE5CDA1FCEB69005A3800D24F4DB66E53F074F811802729733E0040E5C5E5C5C8015F9613937B83F23B278724068018014A00588014005519801EC04B220116CC0402000EAEC03519801A402B30801A802138801400170A0046A800C10001AB37FD8EB805D1C266963E95A4D1A5FF9719FEF7FDB4FB2DB29008CD2BAFA3D005CD31EB4EF2EBE4F4235DF78C66009E80293AE9310D3FCBFBCA440144580273BAEE17E55B66508803C2E0087E630F72BCD5E71B32CCFBBE2800017A2C2803D272BCBCD12BD599BC874B939004B5400964AE84A6C1E7538004CD300623AC6C882600E4328F710CC01C82D1B228980292ECD600B48E0526E506F700760CCC468012E68402324F9668028200C41E8A30E00010D8B11E62F98029801AB88039116344340004323EC48873233E72A36402504CB75006EA00084C7B895198001098D91AE2190065933AA6EB41AD0042626A93135681A400804CB54C0318032200E47B8F71C0001098810D61D8002111B228468000

In [337]:
# Puzzle 1

def parse_literal(bits):
    n = 0
    literal = ""
    while True:
        literal += bits[n+1:n+5]
        if not int(bits[n]): break
        n+=5
    #print(f'Literal found: {int(literal, 2)}')
    return bits[n+5:]


def parse_packet(bits, s=0):

    if bits is None or len(bits) < 6:
        return bits, s
    V = bits[0:3]
    T = bits[3:6]
    #print(f'Version: {int(V, 2)}, Type: {T}')

    s += int(V, 2)

    if T == "100": # Literals, 4
        input = bits[6:]
        rest = parse_literal(input)
        return rest, s
    else:
        I = bits[6]
        if int(I) == 0:
            L = bits[7:7+15]
            length = int(L, 2)
            sub_packets = bits[7+15:7+15+length]
            while sub_packets:
                sub_packets, s = parse_packet(sub_packets, s)
            return bits[7+15+length:], s
        else:
            L = bits[7:7+11]
            number = int(L, 2)
            rest = bits[7+11:]
            for n in range(number):
                rest, s = parse_packet(rest, s)
            return rest, s


parse_packet(bits)

('0', 986)

In [346]:
# Tests
hex_list = [
    "8A004A801A8002F478",
    "620080001611562C8802118E34",
    "C0015000016115A2E0802F182340", #wrong
    "A0016C880162017C3686B18A3D4780",
    "0052E4A00905271049796FB8872A0D25B9FB746893847236200B4F0BCE5194401C9B9E3F9C63992C8931A65A1CCC0D222100511A00BCBA647D98BE29A397005E55064A9DFEEC86600BD002AF2343A91A1CCE773C26600D126B69D15A6793BFCE2775D9E4A9002AB86339B5F9AB411A15CCAF10055B3EFFC00BCCE730112FA6620076268CE5CDA1FCEB69005A3800D24F4DB66E53F074F811802729733E0040E5C5E5C5C8015F9613937B83F23B278724068018014A00588014005519801EC04B220116CC0402000EAEC03519801A402B30801A802138801400170A0046A800C10001AB37FD8EB805D1C266963E95A4D1A5FF9719FEF7FDB4FB2DB29008CD2BAFA3D005CD31EB4EF2EBE4F4235DF78C66009E80293AE9310D3FCBFBCA440144580273BAEE17E55B66508803C2E0087E630F72BCD5E71B32CCFBBE2800017A2C2803D272BCBCD12BD599BC874B939004B5400964AE84A6C1E7538004CD300623AC6C882600E4328F710CC01C82D1B228980292ECD600B48E0526E506F700760CCC468012E68402324F9668028200C41E8A30E00010D8B11E62F98029801AB88039116344340004323EC48873233E72A36402504CB75006EA00084C7B895198001098D91AE2190065933AA6EB41AD0042626A93135681A400804CB54C0318032200E47B8F71C0001098810D61D8002111B228468000E5269324AD1ECF7C519B86309F35A46200A1660A280150968A4CB45365A03F3DDBAE980233407E00A80021719A1B4181006E1547D87C6008E0043337EC434C32BDE487A4AE08800D34BC3DEA974F35C20100BE723F1197F59E662FDB45824AA1D2DDCDFA2D29EBB69005072E5F2EDF3C0B244F30E0600AE00203229D229B342CC007EC95F5D6E200202615D000FB92CE7A7A402354EE0DAC0141007E20C5E87A200F4318EB0C"
]
expected_list = [16, 12, 23, 31, 986]

for hex, expected in zip(hex_list, expected_list):
    print("------------ Test ------------")
    bits = bin(int(hex, 16))[2:].zfill(4*len(hex))
    _, s = parse_packet(bits)
    print(s == expected)


------------ Test ------------
True
------------ Test ------------
True
------------ Test ------------
True
------------ Test ------------
True
------------ Test ------------
True


In [350]:
# Puzzle 2

def parse_literal(bits, value):
    n = 0
    literal = ""
    while True:
        literal += bits[n+1:n+5]
        if not int(bits[n]): break
        n+=5
    #print(f'Literal found: {int(literal, 2)}')
    return bits[n+5:], int(literal, 2) # Return rest and value


def parse_packet(bits, value=0, s=0):
    if bits is None or len(bits) < 6:
        return bits, value, s
    V = bits[0:3]
    T = bits[3:6]
    #print(f'Version: {int(V, 2)}, Type: {T}')

    s += int(V, 2)

    value_list = []
    if T == "100": # Literals, 4
        input = bits[6:]
        rest, value = parse_literal(input, value)
    else: # Operator
        I = bits[6]
        if int(I) == 0:
            L = bits[7:7+15]
            length = int(L, 2)
            sub_packets = bits[7+15:7+15+length]
            while sub_packets:
                sub_packets, value, s = parse_packet(sub_packets, value, s)
                value_list.append(value)
            rest = bits[7+15+length:]
        else:
            L = bits[7:7+11]
            number = int(L, 2)
            rest = bits[7+11:]
            for n in range(number):
                rest, value, s = parse_packet(rest, value, s)
                value_list.append(value)

    if T == "000": # maybe do mapping with functions instead of if/elif
        value = np.sum(value_list)
    elif T == "001":
        value = np.prod(value_list)
    elif T == "010":
        value = np.min(value_list)
    elif T == "011":
        value = np.max(value_list)
    elif T == "101":
        value = int(value_list[0] > value_list[1])
    elif T == "110":
        value = int(value_list[0] < value_list[1])
    elif T == "111":
        value = int(value_list[0] == value_list[1])

    return rest, value, s

#hex = "C200B40A82"
#bits = bin(int(hex, 16))[2:].zfill(4*len(hex))
parse_packet(bits)

('0', 18234816469452, 986)

In [345]:
# Tests
hex_list = [
    "C200B40A82",
    "04005AC33890",
    "880086C3E88112",
    "CE00C43D881120",
    "D8005AC2A8F0",
    "F600BC2D8F",
    "9C005AC2F8F0",
    "9C0141080250320F1802104A08",
]
expected_list = [3, 54, 7, 9, 1, 0, 0, 1]

value = 0
for hex, expected in zip(hex_list, expected_list):
    print("------------ Test ------------")
    bits = bin(int(hex, 16))[2:].zfill(4*len(hex))
    _, value, _ = parse_packet(bits) # empty list is needed here?
    print(value == expected)

------------ Test ------------
True
------------ Test ------------
True
------------ Test ------------
True
------------ Test ------------
True
------------ Test ------------
True
------------ Test ------------
True
------------ Test ------------
True
------------ Test ------------
True
